In [1]:
# import libraries

import numpy as np
import pandas as pd
import csv
import os
from tabulate import tabulate
import glob 
import joblib
from prettytable import PrettyTable
from time import time
from itertools import product

## Data Accessing, Cleaning, and Chunking

In [2]:
def chunk_csv(file_path, chunk_size, output_dir):
    # Extract the file name without extension to use in naming chunks
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)

        chunk_number = 1
        chunk_file = None
        chunk_writer = None
        for i, row in enumerate(reader):
            if i % chunk_size == 0:
                if chunk_file is not None:
                    chunk_file.close()
                chunk_filename = f'{output_dir}/{base_name}_chunk_{chunk_number}.csv'
                chunk_file = open(chunk_filename, 'w', newline='', encoding='utf-8')
                chunk_writer = csv.writer(chunk_file)
                chunk_writer.writerow(headers)
                chunk_number += 1
            chunk_writer.writerow(row)
        if chunk_file is not None:
            chunk_file.close()

def reset():
    df_games = pd.read_csv('../raw_data/raw_games.csv') 
    df_nh = pd.read_csv('../raw_data/raw_necessary_hardware.csv')
    df_oc = pd.read_csv('../raw_data/raw_open_critic.csv')
    df_sn = pd.read_csv('../raw_data/raw_social_networks.csv')
    
    df_games_chunk = pd.read_csv('../chunked_data/clean_games_chunk_1.csv')
    df_nh_chunk = pd.read_csv('../chunked_data/clean_nh_chunk_1.csv')
    df_oc_chunk = pd.read_csv('../chunked_data/clean_oc_chunk_1.csv')
    df_sn_chunk = pd.read_csv('../chunked_data/clean_sn_chunk_1.csv')
    
    df_games = df_games.dropna()
    df_nh = df_nh.dropna()
    df_oc = df_oc.dropna()
    df_sn = df_sn.dropna()
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print(' ')
    print('DATA RESETTED')
    print(' ')
    
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    

    df_games.to_csv('../cleaned_data/clean_games.csv', index=False)
    df_nh.to_csv('../cleaned_data/clean_nh.csv', index=False)
    df_oc.to_csv('../cleaned_data/clean_oc.csv', index=False)
    df_sn.to_csv('../cleaned_data/clean_sn.csv', index=False)
    
    chunk_csv('../cleaned_data/clean_games.csv', 500, '../chunked_data')     # Adjust chunk_size as needed
    chunk_csv('../cleaned_data/clean_nh.csv', 500, '../chunked_data')     # Adjust chunk_size as needed
    chunk_csv('../cleaned_data/clean_oc.csv', 500, '../chunked_data')     # Adjust chunk_size as needed
    chunk_csv('../cleaned_data/clean_sn.csv', 500, '../chunked_data')     # Adjust chunk_size as needed



def memory_usage():
    df_games = pd.read_csv('../raw_data/raw_games.csv') 
    df_nh = pd.read_csv('../raw_data/raw_necessary_hardware.csv')
    df_oc = pd.read_csv('../raw_data/raw_open_critic.csv')
    df_sn = pd.read_csv('../raw_data/raw_social_networks.csv')
    
    df_games_chunk = pd.read_csv('../chunked_data/clean_games_chunk_1.csv')
    df_nh_chunk = pd.read_csv('../chunked_data/clean_nh_chunk_1.csv')
    df_oc_chunk = pd.read_csv('../chunked_data/clean_oc_chunk_1.csv')
    df_sn_chunk = pd.read_csv('../chunked_data/clean_sn_chunk_1.csv')
    
    print(df_games.info(memory_usage='deep'))
    
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_nh.info(memory_usage='deep'))
    
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_oc.info(memory_usage='deep'))
    
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_sn.info(memory_usage='deep'))
    
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_games_chunk.info(memory_usage='deep'))
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_nh_chunk.info(memory_usage='deep'))
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_oc_chunk.info(memory_usage='deep'))
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    
    print(df_sn_chunk.info(memory_usage='deep'))
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    

In [3]:
memory_usage()

reset()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            915 non-null    object
 1   name          915 non-null    object
 2   game_slug     915 non-null    object
 3   price         915 non-null    int64 
 4   release_date  915 non-null    object
 5   platform      783 non-null    object
 6   description   915 non-null    object
 7   developer     712 non-null    object
 8   publisher     707 non-null    object
 9   genres        757 non-null    object
dtypes: int64(1), object(9)
memory usage: 811.2 KB
None
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<class 'pandas.core.frame.DataFrame'>
Index: 1765 entries, 4c81547b81064acfb1902be7b06d63661 to c7372a04d62b4d4bb5b2a95424202e252
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  

# EXEC

In [51]:


import csv
import glob
import joblib
from time import time
from prettytable import PrettyTable

# Parse SQL Query   
def parse_sql_query(sql_query):
    # Initialize condition and from_part variables
    condition = None
    from_part = None
    select_part = None

    # Split the query into SELECT and FROM parts, and optionally WHERE
    if "SELECT" in sql_query:
        # Split SELECT and FROM parts
        select_part, rest_of_query = sql_query.split(" FROM ")
        columns = select_part.replace("SELECT ", "").strip()

        # Check if all columns are requested with a wildcard '*'
        if columns == '*' or columns == 'all':
            columns = None
        else:
            columns = columns.split(", ")

        # Initialize variables for WHERE and ORDER BY clauses
        where_condition = None
        order_by_column = None
        is_ascending = True  # Default sort order

        # Split WHERE (if exists) and ORDER BY parts
        if " WHERE " in rest_of_query:
            from_part, where_part = rest_of_query.split(" WHERE ")
            table_name = from_part.strip()

            if " ORDER BY " in where_part:
                where_condition, order_by_part = where_part.split(" ORDER BY ")
                order_by_column = order_by_part.strip()
            else:
                where_condition = where_part.strip()
        elif " ORDER BY " in rest_of_query:
            table_name, order_by_part = rest_of_query.split(" ORDER BY ")
            order_by_column = order_by_part.strip()
        else:
            table_name = rest_of_query.strip()

        # Check for ascending or descending order
        if order_by_column:
            if " DESC" in order_by_column:
                is_ascending = False
                order_by_column = order_by_column.replace(" DESC", "").strip()
            else:
                order_by_column = order_by_column.replace(" ASC", "").strip()

        # Execute the query based on the parsed components
        select_from_table(table_name, columns, where_condition, order_by_column, is_ascending)
        # print_sorted_data(result)
            
    if "INSERT INTO" in sql_query:
        print("inserting")
        insert_part, values_part = sql_query.split(" VALUES ")
        table_name = insert_part.replace("INSERT INTO ", "").strip()
        values = values_part.strip("()").replace("'","").split(",")
        #calling function to insert into table
        insert_into_table(table_name, values)
        
    if "DELETE" in sql_query:
        delete_part, where_part = sql_query.split("WHERE")
        table_name = delete_part.replace("DELETE FROM",'').strip()
        condition = where_part.strip()
        # calling function to delete from table        
        delete_from_table(table_name, condition)
    
    if "UPDATE" in sql_query: 
        x,y = sql_query.split("SET")
        table_name = x.replace("UPDATE", "").strip()
        setp, condition = y.split("WHERE")
        condition = condition.strip()
        k,v = setp.replace(" ", "").split("=")
        # typecasting setter value to correct type
        if v.isnumeric():
            v = int(v)
        elif v.replace('.', '', 1).isdigit():
            v = float(v)
        else:
            pass    
        setter = (k,v)
        # calling function to update table
        update_table(table_name, setter, condition)

    if "AVG" in sql_query:
        avg_part, from_part = sql_query.split("FROM")
        column_name = avg_part.replace("AVG", "").strip()
        table_name = from_part.strip()
        # calling function to calculate average
        average(table_name, column_name)
        
    if "SUM" in sql_query:
        sum_part, from_part = sql_query.split("FROM")
        column_name = sum_part.replace("SUM", "").strip()
        table_name = from_part.strip()
        # calling function to calculate sum
        sum(table_name, column_name)
        
    if "MIN" in sql_query:
        min_part, from_part = sql_query.split("FROM")
        column_name = min_part.replace("MIN", "").strip()
        table_name = from_part.strip()
        # calling function to calculate minimum
        min(table_name, column_name)
    
    if "MAX" in sql_query:
        max_part, from_part = sql_query.split("FROM")
        column_name = max_part.replace("MAX", "").strip()
        table_name = from_part.strip()
        # calling function to calculate maximum
        max(table_name, column_name)
        
    if "COUNT" in sql_query:
        count_part, from_part = sql_query.split(" FROM ")
        table_name = from_part.strip()
        # Check if a WHERE clause is present
        if from_part and " WHERE " in from_part:
            from_part, where_part = from_part.split(" WHERE ")
            table_name = from_part.strip()
            condition = where_part.strip()
            
        select_columns = count_part.replace("COUNT ", "").strip()
        # Check if all columns are requested with a wildcard '*' or 'all'
        if select_columns == '*' or select_columns == 'all':
            columns = None
        else:
            columns = select_columns
        # calling function to select from table
        count(table_name, columns, condition)

    if "JOIN" in sql_query:
        where_condition = None
        order_by_column = None
        is_ascending = True  # Default sort order
        
        # Split the query into its main components
        parts = sql_query.split(" JOIN ")
        
        # Extract table names
        table1_name = parts[0].strip()
        join_and_rest = parts[1]

        # Extract join condition and the rest of the query
        join_part, rest_of_query = join_and_rest.split(" ON ", 1)
        table2_name = join_part.strip()
        # print(f"table1_name: {table1_name}")
        # print(f"table2_name: {table2_name}")
        

        # Check for WHERE clause
        if " WHERE " in rest_of_query:
            where_part, rest_of_query = rest_of_query.split(" WHERE ", 1)
            where_condition = where_part.strip()

        # Check for ORDER BY clause
        if " ORDER BY " in rest_of_query:
            join_condition, order_by_part = rest_of_query.split(" ORDER BY ", 1)
            
            # order_by_part = rest_of_query.split(" ORDER BY ", 1)[1]
            order_by_column, *order_direction = order_by_part.strip().split()
            # Determine if the sort order is ascending or descending
            is_ascending = order_direction != ['DESC']
        # print(f"join_condition: {join_condition}")
        
        # print(f"where_condition: {where_condition}")
        # print(f"order_by_column: {order_by_column}")
        # print(f"is_ascending: {is_ascending}")
        # Now call the join function with the parsed parameters
        sql_join(table1_name, table2_name, join_condition, where_condition, order_by_column, is_ascending)
        
# selecting from table function which takes table name, columns and condition as input
def select_from_table(table_name, columns, condition, order_by_column=None, is_ascending=True):
    # Find all chunk files for the table
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")

    # Start with an empty list to store selected rows
    selected_rows = []

    # Create a metadata dictionary to store the data types of each column
    metadata = {'type': {}}

    with open(chunk_files[0], 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        if columns is None:
            columns = reader.fieldnames

        _r = next(reader)
        for k, v in _r.items():
            if v.isnumeric():
                metadata['type'][k] = int
            elif v.replace('.', '', 1).isdigit():
                metadata['type'][k] = float
            else:
                metadata['type'][k] = str

    st = time()  # Start time

    # Function to process each chunk file
    def process_chunk(file_path):
        rows = []
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                _lcl = {}
                for k, v in row.items():
                    if metadata['type'][k] == int:
                        _lcl[k] = int(v)
                    elif metadata['type'][k] == float:
                        _lcl[k] = float(v)
                    else:
                        _lcl[k] = v

                if condition is None or eval(condition, {}, _lcl):
                    rows.append({col: row[col] for col in columns})
        return rows

    # Process each chunk file in parallel using joblib
    results = joblib.Parallel(n_jobs=-1)(joblib.delayed(process_chunk)(file_path) for file_path in chunk_files)

    # Flatten the results and store in selected_rows
    for rows in results:
        selected_rows.extend(rows)

    # Sort the data based on the order_by_column
    if order_by_column:
        selected_rows.sort(key=lambda x: metadata['type'][order_by_column](x[order_by_column]), reverse=not is_ascending)

    et = time()  # End time
    print(f"Time taken to process {len(chunk_files)} chunk files: {et - st:.2f} seconds")

    # Print the table using PrettyTable
    table = PrettyTable()
    table.field_names = columns
    for row in selected_rows:
        table.add_row([row[col] for col in columns])

    print(table)
    # result_text_box.value = str(table)
    
    ett = time()  # End time
    print(f"Time taken to print {len(chunk_files)} chunk files: {ett - et:.2f} seconds")

        
def insert_into_table(table_name, values):
    # print("table name:", table_name)
    # print("values:", values)
    
    # Find the latest chunk file for the table
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")

    # Filter out files that don't fit the pattern and find the latest chunk
    valid_chunk_files = [f for f in chunk_files if f.split('_')[-1].split('.')[0].isdigit()]
    if not valid_chunk_files:
        raise FileNotFoundError("No valid chunk files found that match the required naming pattern.")
    # print("valid chunk files: ",valid_chunk_files)
    # latest_chunk_file = max(valid_chunk_files, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    
    latest_chunk_file = valid_chunk_files[-1]           # alternative to above line
    
    print(f"Inserting {values} into {latest_chunk_file}")
    # Append the new values to the latest chunk file
    with open(latest_chunk_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(values)

# delete from table function which takes table name and condition as input
def delete_from_table(table_name, condition):
    # Find all chunk files for the table
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    # Rest of the code...
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    
    metadata = {}
    metadata['type'] = {}
    columns = []
    with open(chunk_files[0], 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        columns = reader.fieldnames

        _r = next(reader)
        for k, v in _r.items():
            if v.isnumeric():
                metadata['type'][k] = int
            elif v.replace('.', '', 1).isdigit():
                metadata['type'][k] = float
            else:
                metadata['type'][k] = str
    
    # Delete rows from each chunk file that satisfy the condition
    for file_path in chunk_files:
        rows_to_keep = []
        rows_to_delete = []
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            total_rows = 0
            for row in reader:
                total_rows += 1
                _lcl = {}
                for k, v in row.items():
                    if metadata['type'][k] == int:
                        _lcl[k] = int(v)
                    elif metadata['type'][k] == float:
                        _lcl[k] = float(v)
                    else:
                        _lcl[k] = v
                if not eval(condition, {}, _lcl):
                    rows_to_keep.append(row)
                else:
                    rows_to_delete.append(row)
                    
            chunk_name = file_path.split('_')[-1].split('.')[0]  
            print(f"rows to delete : {total_rows - len(rows_to_keep)} in chunk #{chunk_name}")
           
        with open(file_path, 'w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=columns)
            writer.writeheader()
            writer.writerows(rows_to_keep)

# update table function which takes table name, setter and condition as input
def update_table(table_name, setter, condition):
    # Find all chunk files for the table
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    # Rest of the code...
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    
    metadata = {}
    metadata['type'] = {}
    columns = []
    with open(chunk_files[0], 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        columns = reader.fieldnames

        _r = next(reader)
        for k, v in _r.items():
            if v.isnumeric():
                metadata['type'][k] = int
            elif v.replace('.', '', 1).isdigit():
                metadata['type'][k] = float
            else:
                metadata['type'][k] = str
    
    # Delete rows from each chunk file that satisfy the condition
    for file_path in chunk_files:
        updated_rows = []
        r_upd = 0
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            total_rows = 0
            for row in reader:
                total_rows += 1
                _lcl = {}
                for k, v in row.items():
                    if metadata['type'][k] == int:
                        _lcl[k] = int(v)
                    elif metadata['type'][k] == float:
                        _lcl[k] = float(v)
                    else:
                        _lcl[k] = v
                if eval(condition, {}, _lcl):
                    # update logic
                    r_upd += 1
                    row[setter[0]] = setter[1]
                    pass  
                updated_rows.append(row)
   
        print(f"rows updated {r_upd}")        
        with open(file_path, 'w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=columns)
            writer.writeheader()
            writer.writerows(updated_rows)
            
# Average function calculates average of a column in a table and takes input table name and column name
def average(table_name, column_name):
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    
    total_sum = 0
    total_count = 0
    
    # Calculate the sum and count of the selected column from each chunk file
    for file_path in chunk_files:
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if column_name in row:
                    value = row[column_name]
                    if value.isdigit():
                        total_sum += int(value)
                        total_count += 1
                    elif value.replace('.', '', 1).isdigit():
                        total_sum += float(value)
                        total_count += 1
    
    # Calculate the average
    if total_count > 0:
        average = total_sum / total_count
        print(f"Average of {column_name} is {average}") 
    else:
        print(f"No values found for {column_name}")
    
# Sum function calculates sum of a column in a table and takes input table name and column name
def sum(table_name, column_name):
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    
    total_sum = 0
    
    # Calculate the sum and count of the selected column from each chunk file
    for file_path in chunk_files:
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if column_name in row:
                    value = row[column_name]
                    if value.isdigit():
                        total_sum += int(value)
                    elif value.replace('.', '', 1).isdigit():
                        total_sum += float(value)
    
    # Calculate the average
    if total_sum > 0:
        print(f"Sum of {column_name} is {total_sum}") 
    else:
        print(f"No values found for {column_name}")
    
# Min function calculates minimum of a column in a table and takes input table name and column name
def min(table_name, column_name):
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    
    min_value = None
    
    # min of the selected column from each chunk file
    for file_path in chunk_files:
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if column_name in row:
                    value = row[column_name]
                    if value.isdigit():
                        value = int(value)
                    elif value.replace('.', '', 1).isdigit():
                        value = float(value)
                    if min_value is None or value < min_value:
                        min_value = value
    
    # Calculate the average
    if min_value is not None:
        print(f"Minimum of {column_name} is {min_value}") 
    else:
        print(f"No values found for {column_name}")
        
# Max function calculates maximum of a column in a table and takes input table name and column name
def max(table_name, column_name):
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")
    max_value = None
    # max of the selected column from each chunk file
    for file_path in chunk_files:
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if column_name in row:
                    value = row[column_name]
                    if value.isdigit():
                        value = int(value)
                    elif value.replace('.', '', 1).isdigit():
                        value = float(value)
                    if max_value is None or value > max_value:
                        max_value = value
    if max_value is not None:
        print(f"Maximum of {column_name} is {max_value}") 
    else:
        print(f"No values found for {column_name}")
        
# Count function calculates count of a value in column in a table and takes input table name, column name and condition
def count(table_name, column_name, condition):
    # Find all chunk files for the table
    chunk_files_pattern = f'../chunked_data/clean_{table_name}_chunk_*.csv'
    chunk_files = glob.glob(chunk_files_pattern)
    if not chunk_files:
        raise FileNotFoundError(f"No files found for the table {table_name}.")

    metadata = {}
    metadata['type'] = {}
    columns = []
    with open(chunk_files[0], 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        columns = reader.fieldnames

        _r = next(reader)
        for k, v in _r.items():
            if v.isnumeric():
                metadata['type'][k] = int
            elif v.replace('.', '', 1).isdigit():
                metadata['type'][k] = float
            else:
                metadata['type'][k] = str
    
    # Delete rows from each chunk file that satisfy the condition
    for file_path in chunk_files:
        count = 0
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            total_rows = 0
            for row in reader:
                total_rows += 1
                _lcl = {}
                for k, v in row.items():
                    if metadata['type'][k] == int:
                        _lcl[k] = int(v)
                    elif metadata['type'][k] == float:
                        _lcl[k] = float(v)
                    else:
                        _lcl[k] = v
                if eval(condition, {}, _lcl):
                    count += 1
        chunk_name = file_path.split('_')[-1].split('.')[0] 
        print(f"count : {count} in chunk #{chunk_name}")

# Join function joins two tables and takes input table names, join column, where condition, order by column and sort order
def sql_join(table1_name, table2_name, join_column, where_condition=None, order_by_column=None, is_ascending=True):
    def read_chunked_data(chunk_files_pattern):
        chunk_files = glob.glob(chunk_files_pattern)
        if not chunk_files:
            raise FileNotFoundError(f"No files found for the pattern {chunk_files_pattern}")

        data = []
        for file_path in chunk_files:
            with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                data.extend(list(reader))
        return data
    # Read data from chunked files
    table1_data = read_chunked_data(f'../chunked_data/clean_{table1_name}_chunk_*.csv')
    table2_data = read_chunked_data(f'../chunked_data/clean_{table2_name}_chunk_*.csv')

    # print(join_column)
    
    # Split the join condition to get the respective column names
    table1_column, table2_column = join_column.split(' = ')
    table1_column = table1_column.split('.')[1]
    table2_column = table2_column.split('.')[1]
    
    # print(f"table1_column: {table1_column}")
    # print(f"table2_column: {table2_column}")
    
    # print(f"First row of {table1_name} data:", table1_data[0] if table1_data else "No data")
    # print(f"First row of {table2_name} data:", table2_data[0] if table2_data else "No data")
    
    # Create a lookup dictionary for the second table using its join column
    table2_lookup = {}
    for row in table2_data:
        key = row[table2_column]  # Use the join column from the second table
        if key in table2_lookup:
            table2_lookup[key].append(row)
        else:
            table2_lookup[key] = [row]

    # Perform the join operation
    joined_data = []
    for row1 in table1_data:
        key = row1[table1_column]  # Use the join column from the first table
        if key in table2_lookup:
            for row2 in table2_lookup[key]:
                combined_row = {**row1, **row2}
                # If WHERE condition is specified, apply it
                if where_condition is None or eval(where_condition, {}, combined_row):
                    joined_data.append(combined_row)
    # print(joined_data)
    
    # Sort the data if ORDER BY clause is specified
    if order_by_column:
        # Handle sorting by converting to the appropriate type
        joined_data.sort(key= lambda x: x[order_by_column.split('.')[-1]], reverse=not is_ascending)

    # Print the joined data if any, or a message if none
    if joined_data:
        print(tabulate(joined_data, headers="keys"))
    else:
        print("No data to print.")

In [5]:
reset()

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 
DATA RESETTED
 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [54]:
import tkinter as tk
from tkinter import scrolledtext
import emoji

emoji_sql_mapping = {
    emoji.emojize(':magnifying_glass_tilted_right:'): 'SELECT',                 # 🔍 works
    emoji.emojize(':file_folder:'): 'FROM',                                     # 📁 works
    emoji.emojize(':red_question_mark:'): 'WHERE',                              # ❓ works
    emoji.emojize(':bar_chart:'): 'ORDER BY',                                   # 📊 works
    emoji.emojize(':cross_mark:'): 'DELETE',                                    # ❌ works
    emoji.emojize(':envelope_with_arrow:'): 'INSERT INTO',                      # 📩 works
    emoji.emojize(':smiling_face_with_halo:'): 'VALUES',                        # 😇 works
    emoji.emojize(':newspaper:'): 'UPDATE',                                     # 📰 works
    emoji.emojize(':check_box_with_check:'): 'SET',                             # ✅ works
    emoji.emojize(':plus:'): 'SUM',                                             # ➕ works
    emoji.emojize(':money_with_wings:'): 'AVG',                                 # 💸 works
    emoji.emojize(':input_numbers:'): 'COUNT',                                  # 🔢 works
    emoji.emojize(':backhand_index_pointing_up:'): 'MAX',                       # 👆 works
    emoji.emojize(':backhand_index_pointing_down:'): 'MIN',                     # 👇 works
    emoji.emojize(':handshake:'): 'JOIN',                                       # 🤝 works
    emoji.emojize(':speaker_low_volume::speaker_high_volume:'): 'ASC',          # 🔉🔊 works
    emoji.emojize(':speaker_high_volume::speaker_low_volume:'): 'DESC',         # 🔊🔉 works
    emoji.emojize(':stop_sign:'): 'ON',                                         # 🛑 works
    # Add more mappings as required
}

# delete_query = "🗑 FROM games WHERE price == 999"


# Parse Emoji Input
def parse_emoji_query(emoji_query):
    # Split the query into tokens
    tokens = emoji_query.split()

    # Translate each token
    translated_tokens = []
    for token in tokens:
        if token in emoji_sql_mapping:
            translated_tokens.append(emoji_sql_mapping[token])
        else:
            translated_tokens.append(token)

    # Reconstruct the query
    sql_query = " ".join(translated_tokens)
    return sql_query

# Function to add emoji to the text box
def add_emoji_to_query(emoji_shortcut):
    query_text_box.insert(tk.END, emoji_shortcut)
    query_text_box.see(tk.END)

# Function to parse the emoji query
def parse_query():
    emoji_query = query_text_box.get("1.0", tk.END)
    emoji_query_emojized = emoji.emojize(emoji_query)
    sql_query = parse_emoji_query(emoji_query_emojized)
    sql_query_box.delete("1.0", tk.END)
    sql_query_box.insert(tk.END, sql_query)
    parse_sql_query(sql_query)

# Set up the main window
root = tk.Tk()
root.title("EmojiQL GUI")


# Define the number of buttons per row
buttons_per_row = 3

# Create a text box for the emoji query with vertical scrollbar
query_text_box = scrolledtext.ScrolledText(root, height=5, width=100)
query_text_box.grid(row=0, column=0, columnspan=buttons_per_row, sticky='ew')
scroll_bar = tk.Scrollbar(root, command=query_text_box.yview)
scroll_bar.grid(row=0, column=buttons_per_row, sticky='ns')
query_text_box['yscrollcommand'] = scroll_bar.set

# Create a text box for the SQL query with vertical scrollbar
sql_query_box = scrolledtext.ScrolledText(root, height=5, width=100)
sql_query_box.grid(row=1, column=0, columnspan=buttons_per_row, sticky='ew')
sql_scroll_bar = tk.Scrollbar(root, command=sql_query_box.yview)
sql_scroll_bar.grid(row=1, column=buttons_per_row, sticky='ns')
sql_query_box['yscrollcommand'] = sql_scroll_bar.set

# Create buttons for each emoji and place them in a grid
for idx, (shortcode, emoji_char) in enumerate(emoji_sql_mapping.items()):
    button = tk.Button(root, text=f"{shortcode} {emoji_char}", command=lambda sc=shortcode: add_emoji_to_query(sc))
    row, col = divmod(idx, buttons_per_row)
    button.grid(row=2 + row, column=col, sticky='nsew', padx=5, pady=5)

# Configure the grid to resize with the window
for i in range(buttons_per_row):
    root.grid_columnconfigure(i, weight=1)
root.grid_rowconfigure(0, weight=0)
root.grid_rowconfigure(1, weight=0)
root.grid_rowconfigure(2, weight=1)

# Create a button to parse the query
parse_button = tk.Button(root, text="Parse Query", command=parse_query, bg="green", fg="white")
parse_button.grid(row=3 + (len(emoji_sql_mapping) // buttons_per_row), column=0, columnspan=buttons_per_row, sticky='ew')

# Create button for reset
reset_button = tk.Button(root, text="Reset", command=reset, bg="red", fg="white")
reset_button.grid(row=4 + (len(emoji_sql_mapping) // buttons_per_row), column=0, columnspan=buttons_per_row, sticky='ew')

# Start the GUI event loop
root.mainloop()


id                                                                                               name                                                                  game_slug                                                             price  release_date              platform                   description                                                                                                                                                                                                                                                                                developer                                                publisher                      genres                                                             operacional_system                                                                                                                                                                processor                                                       memory                 

In [53]:
reset()

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 
DATA RESETTED
 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
